In [ ]:
import sys
sys.path.append('/work/')

from dCSFA_NMF import dCSFA_NMF
import umc_data_tools as umc_dt
import pandas as pd
import pickle
import numpy as np

# how to calculate power/coherence/granger causality
import numpy as np
from scipy.signal import welch, coherence
import statsmodels.api as sm
from spectral_connectivity import Multitaper, Connectivity
import re

INPUT: Pandas dataframe created by Leo that has the LFP data and label data.

OUTPUT: A dictionary of arrays with spectral power density, coherence and granger causality calculated from the LFP data.

In [ ]:
# import data
EPHYS_DATA_FILE = "rce_pilot_2_10_per_trial_spectral_sleap_spikes.pkl"

with open(EPHYS_DATA_FILE,"rb") as f:
    ephys_data = pickle.load(f)

# remove duplicated/redundant columns
# ephys_data_nodup = ephys_data.T.drop_duplicates(keep='first').T
to_drop = []
columns = ephys_data.columns

for i in range(len(columns)):
    for j in range(i + 1, len(columns)):
        if ephys_data.iloc[:, i].equals(ephys_data.iloc[:, j]):
            to_drop.append(columns[j])

# Drop the duplicate columns
ephys_data_nodup = ephys_data.drop(columns=to_drop)

# select which columns to keep
ephys_columns = [
    "current_subject",
    "agent",
    "experiment",
    "in_video_subjects",
    "trial_label",
    'trial_mPFC_lfp_trace',
    'trial_MD_lfp_trace',
    'trial_LH_lfp_trace',
    'trial_BLA_lfp_trace',
    'trial_vHPC_lfp_trace',
    'trial_trace_timestamps',
    'trial_subject_thorax_velocity',
    'trial_agent_thorax_velocity',
]

# get a subset of only the columns of interest
ephys_data_nodup_subset = ephys_data_nodup[ephys_columns]

# drop rows with None in agent column
ephys_data_nodup_subset = ephys_data_nodup_subset.dropna(subset=['agent'])

In [ ]:
cols_to_use = ['tracked_subject',
    'box_number',
    'sleap_name',
    'video_name',
    'current_subject',
    'tone_start_frame',
    'reward_start_frame',
    'tone_stop_frame',
    'condition',
    'competition_closeness',
    'notes',
    'experiment',
    'session_dir',
    'all_subjects',
    'tone_start_timestamp',
    'tone_stop_timestamp',
    'trial_label',
    'cohort',
    'tone_frames',
    'box_1_port_entry_frames',
    'box_2_port_entry_frames',
    'session_path',
    'recording',
    'first_timestamp',
    'last_timestamp',
    'tone_timestamps',
    'box_1_port_entry_timestamps',
    'box_2_port_entry_timestamps',
    'lfp_timestamps',
    'start_frame',
    'stop_frame',
    'in_video_subjects',
    'body_parts',
    'box_top_left',
    'box_top_right',
    'reward_port',
    'box_bottom_left',
    'box_bottom_right',
    'agent',
    'baseline_start_timestamp',
    'baseline_stop_timestamp',
    'baseline_mPFC_lfp_trace',
    'baseline_MD_lfp_trace',
    'baseline_LH_lfp_trace',
    'baseline_BLA_lfp_trace',
    'baseline_vHPC_lfp_trace',
    'baseline_trace_timestamps',
    'trial_mPFC_lfp_trace',
    'trial_MD_lfp_trace',
    'trial_LH_lfp_trace',
    'trial_BLA_lfp_trace',
    'trial_vHPC_lfp_trace',
    'trial_trace_timestamps',
    'baseline_agent_locations',
    'baseline_agent_thorax_to_reward_port',
    'baseline_agent_thorax_velocity',
    'baseline_subject_locations',
    'baseline_subject_thorax_to_reward_port',
    'baseline_subject_thorax_velocity',
    'baseline_video_timestamps',
    'trial_agent_locations',
    'trial_agent_thorax_to_reward_port',
    'trial_agent_thorax_velocity',
    'trial_subject_locations',
    'trial_subject_thorax_to_reward_port',
    'trial_subject_thorax_velocity',
    'trial_video_timestamps',
    'baseline_spike_times',
    'trial_spike_times',
    'baseline_neuron_average_fr',
    'baseline_neuron_average_timestamp',
    'trial_neuron_average_fr',
    'trial_neuron_average_timestamp'
]

export_df = ephys_data_nodup[cols_to_use]
export_df.to_csv('cohort_2_data.csv', index=False)

### Replication of test_data structure

In [ ]:
# initialize dictionary for data in same structure as expected by model
rep_data_dict = {
    "X_lfp" : None,
    "X_psd" : None,
    "X_coh" : None,
    "X_gc" : None,
    "y_mouse" : None,
    "y_task" : None, # this is used for the label of the model
    "Freqs" : None, # frequencies at which to calculate features
    "powFeatures" : None,
    "cohFeatures" : None,
    "gcFeatures" : None,
    "areas" : None
}

# Calculate features from LFP data

In [ ]:
# areas fo the brain under investigation
rep_data_dict["areas"] = sorted([
    'mPFC', # medial Prefrontal Cortex
    'MD', # Mediodorsal Thalamus
    'LH', # Lateral Hypothalamus
    'BLA', # Basolateral Amygdala
    'vHPC' # ventral Hippocampus
])

TOTAL_FREQS = 56 # 56 for full analysis, 0 for all together
# frequencies to calculate power/coherence/granger causality over
rep_data_dict["Freqs"] = np.array(range(1, TOTAL_FREQS+1))

# create power feature names
pow_features = []
for brain_area in rep_data_dict["areas"]:
    for freq in rep_data_dict["Freqs"]:
        pow_f = brain_area + " " + str(freq)
        pow_features.append(pow_f)
rep_data_dict["powFeatures"] = np.array(pow_features)
print("unique power features per frequency")
print(np.sort(np.unique(np.array([i.split(' ')[0] for i in list(rep_data_dict['powFeatures'])]))))

# create coherence feature names
coh_features = []

for brain_area_1 in rep_data_dict["areas"]:
    for brain_area_2 in rep_data_dict["areas"]:
        not_processed_before = brain_area_2 not in rep_data_dict["areas"][:rep_data_dict["areas"].index(brain_area_1)+1]
        if not_processed_before:
            for freq in rep_data_dict["Freqs"]:
                coh_f = brain_area_1 + "-" + brain_area_2 + " " + str(freq)
                coh_features.append(coh_f)
coh_features = pd.Series(coh_features).drop_duplicates().tolist() # remove duplicates in coh_features
coh_features.sort()
rep_data_dict["cohFeatures"] = np.array(coh_features)
print("unique coherence features per frequency")
print(np.sort(np.unique(np.array([i.split(' ')[0] for i in list(rep_data_dict['cohFeatures'])]))))

# create granger causality feature names
gc_features = []
for brain_area_1 in rep_data_dict["areas"]:
    for brain_area_2 in rep_data_dict["areas"]:
        if brain_area_1 != brain_area_2:
            for freq in rep_data_dict["Freqs"]:
                gc_f1 = brain_area_1 + "->" + brain_area_2 + " " + str(freq)
                gc_features.append(gc_f1)
gc_features = pd.Series(gc_features).drop_duplicates().tolist() # remove duplicates in gc_features
rep_data_dict["gcFeatures"] = np.array(gc_features)
print("unique gc features per frequency")
print(np.sort(np.unique(np.array([i.split(' ')[0] for i in list(rep_data_dict['gcFeatures'])]))))

# get mouse labels
rep_data_dict["y_mouse"] = np.array(ephys_data_nodup_subset["current_subject"])

# get trial/task labels
rep_data_dict["y_task"] = np.array(ephys_data_nodup_subset["trial_label"])

# create actual data
# LFP data
lfp_data = []
for brain_area in rep_data_dict["areas"]:
    # how to reshape the data to be in thje correct format
    lfp = ephys_data_nodup_subset['trial_'+ brain_area +'_lfp_trace'].values
    # Ensure all arrays have length 10000
    lfp_trimmed = [arr[:9999] for arr in lfp] # THIS NEEDS TO BE DONE BETTER
    lfp = np.vstack(lfp_trimmed)
    lfp_data.append(lfp)
rep_data_dict["X_lfp"] = np.array(lfp_data)

print("Data structure created")
print("Calculating features")

# power data calculation
pow_data = []
for samp in range(len(rep_data_dict["X_lfp"][0])):
    pow_val_lst = []
    for brain_area_num in range(len(rep_data_dict["areas"])):
        # get the power data for the current brain area and sample
        pow_dt = rep_data_dict["X_lfp"][brain_area_num][samp]
        # Oscillatory power using Welch's method
        fs = 1000 # Sampling frequency in Hz
        desired_freq_resolution = 1
        # overlap deafults to 0.5 
        f, data_psd = welch(
            pow_dt,
            fs, 
            nperseg=fs//desired_freq_resolution
        ) # nperseg=fs to have resolution of 1Hz               
        # select the power for the desired frequency and brain region
        for freq in rep_data_dict["Freqs"]:
            # Find the index of the closest frequency in the PSD output
            idx = np.argmin(np.abs(f - freq))
            pow_val_lst.append(data_psd[idx])
    # append the power data to the list
    pow_data.append(np.array(pow_val_lst))
rep_data_dict["X_psd"] = np.array(pow_data)

print("Power features calculated")

# coherence data calculation
# get the features for each brain area
coh_data = []
for samp in range(len(rep_data_dict["X_lfp"][0])):
    coh_val_lst = []
    for fet in rep_data_dict["cohFeatures"]:
        result = re.split(r'[ \-]+', fet)
        coh_features = [s for s in result if s]
        # get the index of the sample
        area_idx_1 = rep_data_dict["areas"].index(coh_features[0])
        area_idx_2 = rep_data_dict["areas"].index(coh_features[1])
        freq = int(coh_features[2])
        # get data for each brain area combination
        area_data_1 = rep_data_dict["X_lfp"][area_idx_1][samp]
        area_data_2 = rep_data_dict["X_lfp"][area_idx_2][samp]
        # Cross-area synchrony (coherence)
        fs = 1000 # Sampling frequency in Hz
        # overlap defaults to 0.5
        desired_freq_resolution = 1
        f, data_coh = coherence(
            area_data_1, 
            area_data_2, 
            fs,
            nperseg=fs//desired_freq_resolution
        )
        # Find the index of the closest frequency
        idx = np.argmin(np.abs(f - freq))
        coh_val_lst.append(data_coh[idx])
    # append the coherence data to the list
    coh_data.append(np.array(coh_val_lst))
rep_data_dict["X_coh"] = np.array(coh_data)

print("Coherence features calculated")

# create granger casuality feature names
# Parameters for Multitaper
time_halfbandwidth_product = 2  # Reasonable trade-off between time and frequency resolution
time_window_duration = None  # Defaults to the entire length of the signal
time_window_step = None  # Irrelevant when time_window_duration is None
sampling_frequency = 1000  # The signals were sampled at 1000 Hz
gc_data = []
for samp in range(len(rep_data_dict["X_lfp"][0])):
    gc_features = {}
    brain_area_combo_lst = []
    for brain_area_1 in rep_data_dict["areas"]:
        for brain_area_2 in rep_data_dict["areas"]:
            brain_area_combo_12 = brain_area_1 + " -> " + brain_area_2
            brain_area_combo_21 = brain_area_2 + " -> " + brain_area_1
            # check to make sure unecessary combinations are not computed
            different_areas = brain_area_1 != brain_area_2
            combo_12_not_in_list = brain_area_combo_12 not in brain_area_combo_lst
            combo_21_not_in_list = brain_area_combo_21 not in brain_area_combo_lst
            if different_areas and combo_12_not_in_list and combo_21_not_in_list:
                # add brain area combination to list
                brain_area_combo_lst.append(brain_area_combo_12)
                brain_area_combo_lst.append(brain_area_combo_21)
                # get the index of the sample
                area_idx_1 = rep_data_dict["areas"].index(brain_area_1)
                area_idx_2 = rep_data_dict["areas"].index(brain_area_2)
                # get data for each brain area combination
                array1 = rep_data_dict["X_lfp"][area_idx_1][samp]
                array2 = rep_data_dict["X_lfp"][area_idx_2][samp]
                # Combine signals into a time series
                time_series = np.stack((array1, array2), axis=-1)
                multitaper = Multitaper(
                time_series=time_series,
                sampling_frequency=sampling_frequency,
                time_halfbandwidth_product=time_halfbandwidth_product,
                time_window_duration=time_window_duration,
                time_window_step=time_window_step
                )
                # Compute the Fourier coefficients
                fourier_coefficients = multitaper.fft()
                # Initialize the Connectivity analysis
                connectivity = Connectivity(
                    fourier_coefficients=fourier_coefficients,
                    expectation_type='trials_tapers',
                    frequencies=multitaper.frequencies,
                    time=multitaper.time
                )
                # Calculate Granger causality
                granger_result = connectivity.pairwise_spectral_granger_prediction()
                # Extract the Granger causality from array1 to array2 and vice versa
                gc_1_to_2 = granger_result[:, :, 0, 1]
                gc_2_to_1 = granger_result[:, :, 1, 0]
                for freq in rep_data_dict["Freqs"]:
                    # generate gcfeature names
                    gc_name_1_to_2 = brain_area_1 + "->" + brain_area_2 + " " + str(freq)
                    gc_name_2_to_1 = brain_area_2 + "->" + brain_area_1 + " " + str(freq)
                    # Select the frequencies between 1 Hz and 56 Hz
                    freq_indx = np.where(multitaper.frequencies.round(1) == freq)[0]
                    # Extract the relevant frequency components
                    sel_freq_1_to_2 = gc_1_to_2[:, freq_indx]
                    sel_freq_2_to_1 = gc_2_to_1[:, freq_indx]

                    nan_1_to_2 = sel_freq_1_to_2[0][0] == np.nan
                    nan_2_to_1 = sel_freq_2_to_1[0][0] == np.nan
                    nan_1_to_2f = type(sel_freq_1_to_2[0][0]) != np.float64
                    nan_2_to_1f = type(sel_freq_2_to_1[0][0]) != np.float64
                    if nan_1_to_2 or nan_2_to_1 or nan_1_to_2f or nan_2_to_1f:
                        print("samp:", samp, 
                        " - area_1:", brain_area_1, 
                        " - area_2:", brain_area_2, 
                        " - freq:", freq, "\n\n")

                    gc_features[gc_name_1_to_2] = sel_freq_1_to_2[0][0]
                    gc_features[gc_name_2_to_1] = sel_freq_2_to_1[0][0]
    # Order the values in the dictionary to be the same as the features
    gc_val_lst = []
    for fet in rep_data_dict["gcFeatures"]:
        gc_val_lst.append(gc_features[fet])
    # append the coherence data to the list
    gc_data.append(np.array(gc_val_lst))
rep_data_dict["X_gc"] = np.array(gc_data)

# replace nan values with 0 if gc could not be calculated
rep_data_dict["X_gc"] = np.nan_to_num(rep_data_dict["X_gc"], nan=0.0)

print("Granger features calculated")
print("---DONE---")

unique power features per frequency
['BLA' 'LH' 'MD' 'mPFC' 'vHPC']
unique coherence features per frequency
['BLA-LH' 'BLA-MD' 'BLA-mPFC' 'BLA-vHPC' 'LH-MD' 'LH-mPFC' 'LH-vHPC'
 'MD-mPFC' 'MD-vHPC' 'mPFC-vHPC']
unique gc features per frequency
['BLA->LH' 'BLA->MD' 'BLA->mPFC' 'BLA->vHPC' 'LH->BLA' 'LH->MD' 'LH->mPFC'
 'LH->vHPC' 'MD->BLA' 'MD->LH' 'MD->mPFC' 'MD->vHPC' 'mPFC->BLA'
 'mPFC->LH' 'mPFC->MD' 'mPFC->vHPC' 'vHPC->BLA' 'vHPC->LH' 'vHPC->MD'
 'vHPC->mPFC']
Data structure created
Calculating features
Power features calculated
Coherence features calculated
Maximum iterations reached. 0 of 1 converged
Granger features calculated
---DONE---


In [ ]:
# print dimensions of data
for i in rep_data_dict.keys():
    try:
        print(i,":", rep_data_dict[i].shape)
    except:
        print(i,":",rep_data_dict[i])

X_lfp : (5, 194, 9999)
X_psd : (194, 280)
X_coh : (194, 560)
X_gc : (194, 1120)
y_mouse : (194,)
y_task : (194,)
Freqs : (56,)
powFeatures : (280,)
cohFeatures : (560,)
gcFeatures : (1120,)
areas : ['BLA', 'LH', 'MD', 'mPFC', 'vHPC']


### Save data top disk

In [ ]:
# Specify the file name
file_name = 'ephys_data_win_lose.pkl'

# Open the file in write-binary mode and save the dictionary
with open(file_name, 'wb') as file:
    pickle.dump(rep_data_dict, file)

<hr>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d14e702a-7f9c-48a2-8d79-c67c05798ffa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>